# AUTOENCODER IMPLEMENTATION

## Imports

In [ ]:
import tensorflow_datasets as tfds
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# WITH ADDITIONAL METRICS

## Data perparation

In [ ]:
# Loading and Preprocessing Data

(dataset_train, dataset_test), dataset_info = tfds.load(
    'mnist_corrupted/identity',
    split=['train[:2000]', 'test[:2000]'],  
    as_supervised=True,
    with_info=True
)

def dataset_to_numpy(dataset):
    image_list = []
    for image, _ in dataset:
        image_list.append(image.numpy())
    return np.array(image_list)

train_images_np = dataset_to_numpy(dataset_train)
test_images_np = dataset_to_numpy(dataset_test)

# Normalizing images and convert to PyTorch tensors
train_images_tensor = torch.Tensor(train_images_np / 255.0).unsqueeze(1).squeeze(-1)
test_images_tensor = torch.Tensor(test_images_np / 255.0).unsqueeze(1).squeeze(-1)

# Creating DataLoader objects
batch_size = 128
train_data = TensorDataset(train_images_tensor)
test_data = TensorDataset(test_images_tensor)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

## Model definition

In [ ]:
# Constructing the autoencoder with encoder and decoder networks

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


autoencoder = AutoEncoder()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
autoencoder.to(device)

## Main Loop

In [ ]:
# Defining loss function, optimizer, and training loop

loss_function = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

epochs = 30
train_losses = []

for epoch in range(epochs):
    autoencoder.train()
    cumulative_loss = 0.0
    for images, in train_loader:
        images = images.to(device)
        optimizer.zero_grad()
        reconstructed_images = autoencoder(images)
        loss = loss_function(reconstructed_images, images)
        loss.backward()
        optimizer.step()
        cumulative_loss += loss.item() * images.size(0)

    epoch_loss = cumulative_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}')

## Error Calculations

In [ ]:
# Computing reconstruction errors 

def get_reconstruction_errors(model, loader):
    model.eval()
    errors = []
    with torch.no_grad():
        for images, in loader:
            images = images.to(device)
            reconstructed_images = model(images)
            loss = torch.mean((reconstructed_images - images) ** 2, dim=[1, 2, 3])
            errors.extend(loss.cpu().numpy())
    return np.array(errors)


train_recon_errors = get_reconstruction_errors(autoencoder, train_loader)

In [ ]:
# Calculating a dynamic threshold for anomaly detection

def dynamic_threshold_calculation(errors):
    mean_error = np.mean(errors)
    std_error = np.std(errors)
    return mean_error + 2 * std_error

anomaly_threshold = dynamic_threshold_calculation(train_recon_errors)
print(f"Anomaly Detection Threshold: {anomaly_threshold}")


In [ ]:
corruptions = [
    'identity', 'shot_noise', 'impulse_noise', 'glass_blur', 'motion_blur',
    'shear', 'scale', 'rotate', 'brightness', 'translate', 'stripe',
    'fog', 'spatter', 'dotted_line', 'zigzag'
]

all_predictions = []
all_ground_truth = []
all_recon_errors = []

for corruption_type in corruptions:
    print(f"Evaluating on corruption type: {corruption_type}")
    dataset_test, dataset_info = tfds.load(
        f'mnist_corrupted/{corruption_type}',
        split='test[:2000]',
        as_supervised=True,
        with_info=True
    )

    test_images_np = dataset_to_numpy(dataset_test)
    test_images_tensor = torch.Tensor(test_images_np / 255.0).unsqueeze(1).squeeze(-1)
    test_loader = DataLoader(TensorDataset(test_images_tensor), batch_size=batch_size, shuffle=False)

    recon_errors = get_reconstruction_errors(autoencoder, test_loader)
    all_recon_errors.extend(recon_errors)

    if corruption_type == 'identity':
        ground_truth_labels = np.zeros_like(recon_errors)
    else:
        ground_truth_labels = np.ones_like(recon_errors)

    predicted_labels = recon_errors > anomaly_threshold
    all_predictions.extend(predicted_labels)
    all_ground_truth.extend(ground_truth_labels)


## Evaluation and Plotting

In [ ]:
# Calculating and displaying confusion matrix

all_predictions = np.array(all_predictions)
all_ground_truth = np.array(all_ground_truth)
conf_matrix = confusion_matrix(all_ground_truth, all_predictions)

TN, FP, FN, TP = conf_matrix.ravel()
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"TN: {TN}, FP: {FP}, FN: {FN}, TP: {TP}")


In [ ]:
# Plot reconstruction error distribution
plt.figure(figsize=(12, 6))
plt.title('Reconstruction Error Distribution')
sns.histplot(all_recon_errors, bins=100, kde=True, color='blue')
plt.axvline(anomaly_threshold, color='red', linestyle='--', label='Threshold')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Original vs reconstructed images
for images, in train_loader:
    images = images.to(device)
    reconstructed_images = autoencoder(images)
    break  

plt.figure(figsize=(12, 6))
plt.suptitle('Original vs Reconstructed Images', fontsize=16)
plt.show()

In [ ]:
# original images
for i in range(6):
    plt.subplot(2, 6, i + 1)
    plt.imshow(images[i].cpu().detach().squeeze(), cmap='gray')
    plt.title('Original')
    plt.axis('off')

# reconstructed images
for i in range(6):
    plt.subplot(2, 6, i + 7)
    plt.imshow(reconstructed_images[i].cpu().detach().squeeze(), cmap='gray')
    plt.title('Reconstructed')
    plt.axis('off')

plt.show()

In [ ]:
# training loss over epochs
plt.figure(figsize=(12, 6))
plt.title('Training Loss Over Epochs')
plt.plot(range(1, epochs + 1), train_losses, marker='o', linestyle='-', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.show()

# confusion matrix for anomalies
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix for Anomalies')
plt.show()


# WITHOUT ANY ADDITIONAL METRICS

## Data perparation

In [ ]:
# Loading dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist_corrupted/identity',  
    split=['train[:2000]', 'test[:2000]'],  
    as_supervised=True,
    with_info=True
)


def to_numpy(ds):
    images = []
    for img, _ in ds:  
        images.append(img.numpy())
    return np.array(images)

train_images = to_numpy(ds_train)
test_images = to_numpy(ds_test)

# Normalizing images and converting to PyTorch tensors
train_images = torch.Tensor(train_images / 255.0).unsqueeze(1).squeeze(-1) 
test_images = torch.Tensor(test_images / 255.0).unsqueeze(1).squeeze(-1)

# Creating DataLoaders
batch_size = 128
train_dataset = TensorDataset(train_images)
test_dataset = TensorDataset(test_images)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



## Model definition

In [ ]:
# Defining autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialization
model = Autoencoder()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



## Training

In [ ]:
# Defining loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, in train_loader:
        images = images.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

## Error Calculations

In [ ]:
# Computing reconstruction errors 
def compute_reconstruction_error(model, loader):
    model.eval()
    errors = []
    with torch.no_grad():
        for images, in loader:
            images = images.to(device)
            outputs = model(images)
            loss = torch.mean((outputs - images) ** 2, dim=[1, 2, 3])
            errors.extend(loss.cpu().numpy())
    return np.array(errors)


train_reconstruction_errors = compute_reconstruction_error(model, train_loader)

# Dynamic threshold calculation
def calculate_dynamic_threshold(losses):
    mean_loss = np.mean(losses)
    std_loss = np.std(losses)
    dynamic_threshold = mean_loss + 2 * std_loss  # Adaptive threshold using z-scores
    return dynamic_threshold

# Setting an initial anomaly detection threshold based on the training loss distribution
threshold = calculate_dynamic_threshold(train_reconstruction_errors)
print(f"Initial anomaly detection threshold: {threshold}")

## Evaluation and Plotting

In [ ]:
# Evaluating across all corruption types
corruption_types = [
    'identity', 'shot_noise', 'impulse_noise', 'glass_blur', 'motion_blur',
    'shear', 'scale', 'rotate', 'brightness', 'translate', 'stripe',
    'fog', 'spatter', 'dotted_line', 'zigzag'
]

all_predictions = []
all_ground_truth = []

for corruption in corruption_types:
    print(f"Evaluating corruption type: {corruption}")
    ds_test, ds_info = tfds.load(
        f'mnist_corrupted/{corruption}',
        split='test[:2000]',  
        as_supervised=True,
        with_info=True
    )

    test_images = to_numpy(ds_test)
    test_images = torch.Tensor(test_images / 255.0).unsqueeze(1).squeeze(-1)
    test_loader = DataLoader(TensorDataset(test_images), batch_size=batch_size, shuffle=False)

    # reconstruction errors
    reconstruction_errors = compute_reconstruction_error(model, test_loader)

    # Creating ground truth labels
    if corruption == 'identity':
        ground_truth = np.zeros_like(reconstruction_errors)  # Normal
    else:
        ground_truth = np.ones_like(reconstruction_errors)   # Anomalous

    # predictions
    predictions = reconstruction_errors > threshold
    all_predictions.extend(predictions)
    all_ground_truth.extend(ground_truth)

# confusion matrix
all_predictions = np.array(all_predictions)
all_ground_truth = np.array(all_ground_truth)
cm = confusion_matrix(all_ground_truth, all_predictions)

# Extract TN, TP, FN, FP
TN, FP, FN, TP = cm.ravel()

print(f"Confusion Matrix:\n{cm}")
print(f"TN: {TN}, FP: {FP}, FN: {FN}, TP: {TP}")